In [78]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests
import json 
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time 
from datetime import datetime

In [79]:
# IF YOU ARE SELECTING NIFTY ,BANKNIFTY,MIDCAP  SELECT SEW[0] FOR INDICES ELSE FOR STOCK SELECT SWG[1] 
symbol="NIFTY"
select_swg=["indices","equities"]
seg=select_swg[0]
EX="28-Mar-2024" #enter the expiry date 

In [80]:

chrome_options = Options()


headers = {
    ":authority":"www.nseindia.com",
    ":method":"GET",
    #":path":f"/api/option-chain-equities?symbol={symbol}",
    ":scheme":"https",
    "Accept":"*/*",
    "Accept-Encoding":"gzip, deflate, br" ,
    "Accept-Language":"en-GB,en-US;q=0.9,en;q=0.8,hi;q=0.7",
    "Referer":"https://www.nseindia.com/option-chain",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
    
}

for key, value in headers.items():
    chrome_options.add_argument(f"--header={key}:{value}")


driver = webdriver.Chrome(options=chrome_options)


driver.get("https://www.nseindia.com/option-chain")

driver.get(f"https://www.nseindia.com/api/option-chain-{seg}?symbol={symbol}")
#driver.get(f"https://www.nseindia.com/api/option-chain-equities?symbol={symbol}")
json_content = driver.page_source
soup = BeautifulSoup(json_content, 'html.parser')

In [81]:
pre_tag = soup.find('pre')
if pre_tag:
    json_content = pre_tag.string.strip()
    print("works")
    driver.close()
    data_dict = json.loads(json_content)
else:
    print("No <pre> tag found in the HTML content.")

works


In [83]:
a=data_dict['records']['data']

In [84]:
ocdata=[]

In [85]:
for i in a:
    for j,k in i.items():
        if j=="CE" or j=="PE":
            info=k
            info["instrumentTyoe"]=j
            ocdata.append(info)


In [86]:
print(len(ocdata))

1380


In [87]:
df=pd.DataFrame(ocdata)
df.drop(columns=['identifier'],inplace=True)

In [88]:
df=df.sort_values(by='expiryDate')

In [89]:
df.reset_index(drop=True,inplace=True)

In [90]:
df['expiryDate']=pd.to_datetime(df['expiryDate'])

In [91]:
df

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,21950,2024-03-07,NIFTY,191.0,-40,-17.316017,2196,24.41,619.20,256.25,70.602011,5200,3900,1150,475.55,1500,617.55,22217.45,CE
1,21250,2024-03-07,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,3250,4150,1500,882.85,1500,1140.05,22217.45,CE
2,21250,2024-03-07,NIFTY,370.0,370,0.000000,1789,16.19,19.00,-149.85,-88.747409,14500,3650,750,19.00,1800,23.05,22217.45,PE
3,23450,2024-03-07,NIFTY,207.0,-4,-1.895735,195,13.58,6.95,1.50,27.522936,39500,8900,400,5.85,1800,8.75,22217.45,CE
4,22300,2024-03-07,NIFTY,4237.0,1042,32.613459,19468,14.84,259.00,76.25,41.723666,13400,4300,50,256.00,50,259.00,22217.45,CE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,24000,2026-12-31,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,600,0,250,550.05,0,0.00,22217.45,PE
1376,25000,2026-12-31,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,600,0,250,725.05,0,0.00,22217.45,PE
1377,14000,2026-12-31,NIFTY,19.0,0,0.000000,0,0.00,500.00,0.00,0.000000,800,0,750,20.95,0,0.00,22217.45,PE
1378,26000,2026-12-31,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,350,0,350,3.00,0,0.00,22217.45,CE


In [92]:
df['expiryDate']==EX

0       False
1       False
2       False
3       False
4       False
        ...  
1375    False
1376    False
1377    False
1378    False
1379    False
Name: expiryDate, Length: 1380, dtype: bool

In [93]:
df[df['instrumentTyoe']=='CE']

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,21950,2024-03-07,NIFTY,191.0,-40,-17.316017,2196,24.41,619.20,256.25,70.602011,5200,3900,1150,475.55,1500,617.55,22217.45,CE
1,21250,2024-03-07,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,3250,4150,1500,882.85,1500,1140.05,22217.45,CE
3,23450,2024-03-07,NIFTY,207.0,-4,-1.895735,195,13.58,6.95,1.50,27.522936,39500,8900,400,5.85,1800,8.75,22217.45,CE
4,22300,2024-03-07,NIFTY,4237.0,1042,32.613459,19468,14.84,259.00,76.25,41.723666,13400,4300,50,256.00,50,259.00,22217.45,CE
5,22400,2024-03-07,NIFTY,2796.0,1822,187.063655,12690,14.38,204.65,60.25,41.724377,21950,2950,250,204.05,100,206.60,22217.45,CE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335,23500,2024-02-29,NIFTY,54033.0,27204,101.397741,107319,16.16,1.30,-0.75,-36.585366,1417750,154450,6950,1.30,2500,1.35,22217.45,CE
1358,23000,2027-12-30,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,50,0,50,2710.00,0,0.00,22217.45,CE
1361,19000,2026-12-31,NIFTY,4.0,0,0.000000,1,0.00,6000.00,1001.00,20.024005,50,0,50,4999.20,0,0.00,22217.45,CE
1374,27000,2026-12-31,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,350,0,350,3.00,0,0.00,22217.45,CE


In [94]:
df[df['instrumentTyoe']=='PE']

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
2,21250,2024-03-07,NIFTY,370.0,370,0.000000,1789,16.19,19.00,-149.85,-88.747409,14500,3650,750,19.00,1800,23.05,22217.45,PE
6,23450,2024-03-07,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,1500,3250,1500,691.50,1500,1350.95,22217.45,PE
7,22450,2024-03-07,NIFTY,395.0,227,135.119048,396,12.71,302.30,-131.20,-30.265283,1750,2100,1500,270.05,300,313.05,22217.45,PE
8,20550,2024-03-07,NIFTY,85.0,85,0.000000,98,19.11,4.05,-39.30,-90.657439,18850,150,2400,3.70,100,6.00,22217.45,PE
12,21850,2024-03-07,NIFTY,1188.0,871,274.763407,3894,14.52,86.10,-70.25,-44.931244,2400,15550,300,57.65,50,110.00,22217.45,PE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,15000,2026-12-31,NIFTY,44.0,0,0.000000,0,0.00,118.90,0.00,0.000000,1200,0,1200,23.30,0,0.00,22217.45,PE
1375,24000,2026-12-31,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,600,0,250,550.05,0,0.00,22217.45,PE
1376,25000,2026-12-31,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,600,0,250,725.05,0,0.00,22217.45,PE
1377,14000,2026-12-31,NIFTY,19.0,0,0.000000,0,0.00,500.00,0.00,0.000000,800,0,750,20.95,0,0.00,22217.45,PE


In [95]:
ce_data=df[(df['expiryDate']==EX) & (df['instrumentTyoe']=="CE")]

In [96]:
ce_data.reset_index(drop=True,inplace=True)

In [97]:
ce_data.head(5)

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,23200,2024-03-28,NIFTY,3358.0,128,3.962848,9920,12.81,98.65,19.75,25.031686,11950,3200,250,98.00,550,104.85,22217.45,CE
1,19950,2024-03-28,NIFTY,0.0,0,0.000000,0,0.00,2257.75,0.00,0.000000,6050,2750,1500,2049.65,1000,2531.00,22217.45,CE
2,23150,2024-03-28,NIFTY,433.0,-58,-11.812627,728,12.80,108.35,21.55,24.827189,12050,650,600,80.40,300,117.15,22217.45,CE
3,20000,2024-03-28,NIFTY,11315.0,1870,19.798835,3316,17.34,2414.90,196.50,8.857735,8650,6150,150,2403.40,50,2411.70,22217.45,CE
4,19900,2024-03-28,NIFTY,7.0,0,0.000000,3,0.00,2332.85,382.85,19.633333,6050,4000,1500,2401.75,1250,2646.90,22217.45,CE


In [98]:
ce_data.columns

Index(['strikePrice', 'expiryDate', 'underlying', 'openInterest',
       'changeinOpenInterest', 'pchangeinOpenInterest', 'totalTradedVolume',
       'impliedVolatility', 'lastPrice', 'change', 'pChange',
       'totalBuyQuantity', 'totalSellQuantity', 'bidQty', 'bidprice', 'askQty',
       'askPrice', 'underlyingValue', 'instrumentTyoe'],
      dtype='object')

In [99]:
pe_data=df[(df['expiryDate']==EX) & (df['instrumentTyoe']=="PE")]

In [100]:
pe_data.reset_index(drop=True,inplace=True)

In [101]:
pe_data.head(5)

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,20000,2024-03-28,NIFTY,55659.0,3357,6.418493,23441,19.88,15.65,-5.55,-26.179245,48100,84400,900,15.10,250,15.40,22217.45,PE
1,23150,2024-03-28,NIFTY,22.0,0,0.000000,0,0.00,1092.10,0.00,0.000000,5100,6050,1500,692.30,1500,1089.75,22217.45,PE
2,19900,2024-03-28,NIFTY,4674.0,191,4.260540,1823,19.91,12.75,-4.55,-26.300578,15300,7800,1000,11.85,3450,15.00,22217.45,PE
3,23200,2024-03-28,NIFTY,87.0,4,4.819277,25,14.30,890.00,-131.40,-12.864696,4700,4800,100,851.45,250,902.70,22217.45,PE
4,19950,2024-03-28,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,13500,0,2400,10.10,0,0.00,22217.45,PE


In [102]:
ce_data.head(1)

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,23200,2024-03-28,NIFTY,3358.0,128,3.962848,9920,12.81,98.65,19.75,25.031686,11950,3200,250,98.0,550,104.85,22217.45,CE


In [103]:
ce_data.iloc[-1]

strikePrice                            21950
expiryDate               2024-03-28 00:00:00
underlying                             NIFTY
openInterest                           293.0
changeinOpenInterest                      19
pchangeinOpenInterest               6.934307
totalTradedVolume                        673
impliedVolatility                      14.05
lastPrice                             666.45
change                                108.55
pChange                            19.456892
totalBuyQuantity                        5050
totalSellQuantity                       4300
bidQty                                   600
bidprice                              654.75
askQty                                    50
askPrice                               679.4
underlyingValue                     22217.45
instrumentTyoe                            CE
Name: 99, dtype: object

In [104]:
ce_data["impliedVolatility"].iloc[-1]

14.05

In [105]:
import numpy as np
from scipy.stats import norm

def black_scholes(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

def calculate_option_price(underlying_price, strike_price, time_to_expiration, risk_free_rate, implied_volatility):
    """
    Calculate option price using the Black-Scholes formula.
    """
    option_price = black_scholes(underlying_price, strike_price, time_to_expiration, risk_free_rate, implied_volatility)
    return option_price

def calculate_gamma(option_price_up, option_price_down, underlying_price, increment=0.01):
    """
    Calculate gamma of an option using numerical differentiation.
    """
    delta_price = 2 * increment
    gamma = (option_price_up + option_price_down - 2 * option_price_up) / (underlying_price ** 2 * delta_price)
    return gamma

def calculate_delta(option_price_up, option_price_down, increment=0.01):
    """
    Calculate delta of an option using numerical differentiation.
    """
    delta_price = 2 * increment
    delta_option = (option_price_up - option_price_down) / delta_price
    return delta_option

def calculate_theta(option_price_up, option_price_down, time_to_expiration, increment=1):
    """
    Calculate theta of an option using numerical differentiation.
    """
    theta = (option_price_up - option_price_down) / increment
    return theta

def calculate_vega(option_price_up, option_price_down, implied_volatility, increment=0.01):
    """
    Calculate vega of an option using numerical differentiation.
    """
    vega = (option_price_up - option_price_down) / increment
    return vega

# Example usage
underlying_price =ce_data['underlyingValue'].iloc[-1]
strike_price = ce_data['strikePrice'].iloc[-1]
today = datetime.now()
expiry_date=ce_data['expiryDate'].iloc[-1]
time_to_expiration = (expiry_date - today).days / 365.0

risk_free_rate = 0.07
implied_volatility =ce_data['impliedVolatility'].iloc[-1]
increment = 0.01

option_price_up = calculate_option_price(underlying_price + increment, strike_price, time_to_expiration, risk_free_rate, implied_volatility)
option_price_down = calculate_option_price(underlying_price - increment, strike_price, time_to_expiration, risk_free_rate, implied_volatility)
option_price = calculate_option_price(underlying_price, strike_price, time_to_expiration, risk_free_rate, implied_volatility)

gamma = calculate_gamma(option_price_up, option_price_down, underlying_price, increment)
delta = calculate_delta(option_price_up, option_price_down, increment)
theta = calculate_theta(option_price_up, option_price_down, time_to_expiration)
vega = calculate_vega(option_price_up, option_price_down, implied_volatility, increment)

print("Option Price:", option_price)
print("Gamma:", gamma)
print("Delta:", delta)
print("Theta:", theta)
print("Vega:", vega)


Option Price: 21454.46153045923
Gamma: -1.991138722471196e-09
Delta: 0.9828561089307186
Theta: 0.019657122178614372
Vega: 1.9657122178614372
